In [4]:
# import essential libraries 
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
from shapely import wkb

In [5]:
parquet_file = "Churu_height_NEW_GHSL_SMOD_elevations.parquet"
target_df = pd.read_parquet(parquet_file)

In [6]:
if target_df['geometry'].dtype == object:
    target_df['geometry'] = target_df['geometry'].apply(wkb.loads)

In [7]:
target_gdf = gpd.GeoDataFrame(target_df, geometry='geometry', crs="EPSG:4326")

In [8]:
target_gdf = target_gdf.to_crs("EPSG:3857")

In [9]:
for dist in [50, 100, 250, 500]:
    target_gdf[f"building_density_{dist}"] = 0


In [11]:
from tqdm import tqdm  

batch_size = 1000
results = []

for i in tqdm(range(0, len(target_gdf), batch_size), desc="Processing buildings", unit="batch"):
    #print(f"Processing batch {i}...")
    target_batch = target_gdf.iloc[i:i+batch_size].copy()

    for dist in [50, 100, 250, 500]:
        target_batch[f"building_density_{dist}"] = 0

        buffer_batch = target_batch.copy()
        buffer_batch["geometry"] = buffer_batch.geometry.buffer(dist)

        joined = gpd.sjoin(target_gdf, buffer_batch, how="inner", predicate="within")
        counts = joined.groupby("index_right").size()

        target_batch.loc[counts.index, f"building_density_{dist}"] = counts.values - 1

    results.append(target_batch)

# concatenate all results
target_gdf = pd.concat(results)
print(f"Processing complete...!")


Processing buildings: 100%|██████████| 1482/1482 [35:52<00:00,  1.45s/batch] 


Processing complete...!


In [12]:
target_gdf = target_gdf.to_crs("EPSG:4326")
print(target_gdf.head())

   perimeter_in_meters  building_faces bf_source  confidence  \
0            10.003668               4    google      0.7370   
1            14.526151               4    google      0.7366   
2            28.127545               4    google      0.7958   
3            25.103799               4    google      0.8284   
4            26.080669               4    google      0.7482   

                                            geometry  longitude   latitude  \
0  POLYGON ((74.46928 28.43321, 74.46928 28.43323...  74.469271  28.433219   
1  POLYGON ((74.46927 28.43434, 74.46927 28.43437...  74.469250  28.434354   
2  POLYGON ((74.4668 28.43541, 74.46678 28.43548,...  74.466757  28.435439   
3  POLYGON ((74.46704 28.43538, 74.46703 28.43542...  74.466999  28.435397   
4  POLYGON ((74.46708 28.43545, 74.46708 28.43549...  74.467039  28.435467   

                                     id  area_in_meters  height_mean  ...  \
0  74.46927144167354:28.433219462727678        6.254586     0.000000 

In [13]:
target_gdf.to_parquet("Churu_height_NEW_GHSL_SMOD_elevations_BDensity.parquet")